In [1]:
%matplotlib inline

In [2]:
import cv2

In [3]:
import numpy as np

In [4]:
import matplotlib
import matplotlib.pyplot as plt

In [5]:
import os

In [6]:
small_regions = [
    (91, 39),
    (212, 39),
    (333, 39),
    (455, 39),
    (91, 120),
    (212, 120),
    (333, 120),
    (455, 120),
    (91, 201),
    (212, 201),
    (333, 201),
    (455, 201),
    (91, 282),
    (212, 282),
    (333, 282),
    (455, 282),
]
small_region_size = (70, 70)
small_size = (600, 400)

large_regions = [
    (114, 59),
    (276, 59),
    (438, 59),
    (599, 59),
    (114, 184),
    (276, 184),
    (438, 184),
    (599, 184),
    (114, 309),
    (276, 309),
    (438, 309),
    (599, 309),
    (114, 435),
    (276, 435),
    (438, 435),
    (599, 435),
]
large_region_size = (108, 108)
large_size = (800, 600)

small_pad = 15
large_pad = 70
font_space = 50

frame_size = ((2+4)*small_pad + 3*large_pad + 2*4*large_region_size[0],
              font_space + 2*small_pad + 3*large_pad + 4*large_region_size[1])

In [7]:
output_file = "autoencoder_movie.mp4"
fourcc = 0x20
frame_rate = 25
stride = 2
repeat_last_frame = 10 * 25
repeat_first_frame = 1 * 25
skip_to_end_after = 950

In [8]:
def get_cells(img, regions, region_size):
    cells = []
    (dx, dy) = region_size
    for (x, y) in regions:
        cell = img[y:y+dy, x:x+dx, 0:3]
        cells.append(cv2.resize(cell, large_region_size))
    return cells

In [9]:
golden = get_cells(plt.imread("test_golden.png"),
                   small_regions,
                   small_region_size)

In [10]:
def compose_frame(img, regions, region_size):
    frame = np.ones((frame_size[1], frame_size[0], 3), dtype='float32')
    cells = get_cells(img, regions, region_size)
    (dx, dy) = large_region_size
    # Place golden
    for y in xrange(4):
        for x in xrange(4):
            ox = small_pad + x*(2*dx + small_pad + large_pad)
            oy = font_space + small_pad + y*(dy + large_pad)
            cell = golden[y*4 + x]
            frame[oy:oy+dy, ox:ox+dx] = cell
    # Place reconstructions
    for y in xrange(4):
        for x in xrange(4):
            ox = 2*small_pad + dx + x*(2*dx + small_pad + large_pad)
            oy = font_space + small_pad + y*(dy + large_pad)
            cell = cells[y*4 + x]
            frame[oy:oy+dy, ox:ox+dx] = cell
            
    frame = (255*frame).astype('uint8')
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    # Place text
    black = (0, 0, 0)
    for y in xrange(4):
        for x in xrange(4):
            oy = y*(dy + large_pad)
            ox = x*(2*dx + small_pad + large_pad)
            cv2.putText(frame, "Original", (ox + small_pad + 25,
                                            oy + font_space),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, black)
            cv2.putText(frame, "Reconstruction", (ox + 2* small_pad + dx - 5,
                                                  oy + font_space),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, black)
    
    return frame
    

In [11]:
frame = compose_frame(plt.imread("images_in_order/img_999.png"), large_regions, large_region_size)
#frame = (255*frame).astype('uint8')
cv2.imwrite("TEST.png", frame)

True

In [12]:
def process_frame(filename):
    img = plt.imread(filename)
    if (img.shape[1], img.shape[0]) == small_size:
        frame = compose_frame(img, small_regions, small_region_size)
    elif (img.shape[1], img.shape[0]) == large_size:
        frame = compose_frame(img, large_regions, large_region_size)
    else:
        assert(False)
    return frame

In [13]:
def make_video(output_file, fourcc, frame_rate, stride, repeat_first_frame, repeat_last_frame,
               skip_to_end_after=None, prefix="images_in_order/"):
    writer = cv2.VideoWriter(output_file, fourcc, frame_rate, frame_size)
    i = 0
    last = None
    while True:
        name = os.path.join(prefix, "img_%d.png" % i)
        if not os.path.exists(name):
            break
        last = process_frame(name)
        if skip_to_end_after is None or i <= skip_to_end_after:
            writer.write(last)
        if i == 0:
            for _ in xrange(repeat_first_frame - 1):
                writer.write(last)
        i += stride
    assert(last is not None)
    for _ in xrange(repeat_last_frame - 1):
        writer.write(last)
    writer.release()

In [14]:
make_video(output_file, fourcc, frame_rate, stride, repeat_first_frame, repeat_last_frame, skip_to_end_after)